In [ ]:
### helper functions

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.preprocessing import normalize
import os
import numpy as np
from typing import List
import pandas as pd
import ast
import csv


def get_documents() -> pd.DataFrame:
    docs_original = pd.read_csv("data/DRAGONball/en/docs.csv", usecols=["doc_id", "domain", "content"])
    docs_manipulated_single_textual = pd.read_csv(
        "data/additional_data/docs/textual_manipulations_result.csv",
        usecols=["doc_id", "domain", "content", "original_doc_id"],
        dtype={"original_doc_id": "Int64"},
    )
    docs_manipulated_single_textual["original_doc_id"] = docs_manipulated_single_textual["original_doc_id"].apply(
        lambda i: [i] if pd.notna(i) else []
    )
    docs_manipulated_single_textual.rename(columns={"original_doc_id": "original_doc_ids"}, inplace=True)

    docs_manipulated_single_tabular = pd.read_csv(
        "data/additional_data/docs/tabular_manipulations_result.csv",
        usecols=["doc_id", "domain", "content", "original_doc_ids"],
        converters={"original_doc_ids": ast.literal_eval},
    )

    docs_manipulated_multi_textual = pd.read_csv(
        "data/additional_data/docs/multi_textual_manipulations.csv",
        usecols=["doc_id", "domain", "content", "original_doc_id"],
        dtype={"original_doc_id": "Int64"},
    )
    docs_manipulated_multi_textual["original_doc_id"] = docs_manipulated_multi_textual["original_doc_id"].apply(
        lambda i: [i] if pd.notna(i) else []
    )
    docs_manipulated_multi_textual.rename(columns={"original_doc_id": "original_doc_ids"}, inplace=True)

    print(f"# original docs: {len(docs_original)}")
    print(f"# manipulated textual docs: {len(docs_manipulated_single_textual)}")
    print(f"# manipulated tabular docs: {len(docs_manipulated_single_tabular)}")
    print(f"# manipulated textual multi docs: {len(docs_manipulated_multi_textual)}")

    return pd.concat(
        [
            docs_original,
            docs_manipulated_single_textual,
            docs_manipulated_multi_textual,
            docs_manipulated_single_tabular,
        ],
        sort=False,
    )


documents = get_documents()

# original docs: 108
# manipulated textual docs: 30
# manipulated tabular docs: 3
# manipulated textual multi docs: 30


In [45]:
### Get embeddings
from openai import OpenAI, RateLimitError
from dotenv import load_dotenv
from tenacity import retry, retry_if_exception_type, wait_random, stop_after_attempt

load_dotenv("/Users/leon/.env")
client = OpenAI()

FILEPATH = "data/additional_data/docs/embeddings.csv"
FIELDNAMES = ["doc_id", "content", "embedding"]


def get_embeddings(input: str) -> List[float]:
    print("DEBUG: Getting embedding from OpenAI")
    embedding = client.embeddings.create(model="text-embedding-3-small", input=input, encoding_format="float")
    print(f"DEBUG: Got embedding. Usage: {embedding.usage.total_tokens} total tokens.")
    return embedding.data[0].embedding

@retry(
    retry=retry_if_exception_type(RateLimitError),
    wait=wait_random(min=30, max=60),
    stop=stop_after_attempt(6),
)
def get_and_save_emebeddings(row) -> None:
    docs_processed = pd.read_csv(FILEPATH, usecols=["doc_id"])["doc_id"].to_list()
    if row["doc_id"] in docs_processed:
        print(f"Doc with ID '{row["doc_id"]}' has been processed before.")
        return 
    
    embedding = get_embeddings(row["content"])
    with open(FILEPATH, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writerow({
            "doc_id": row["doc_id"],
            "content": row["content"],
            "embedding": embedding
        })


In [46]:
_ = documents.apply(get_and_save_emebeddings, axis=1)

print("Done")

Doc with ID '40' has been processed before.
Doc with ID '41' has been processed before.
Doc with ID '42' has been processed before.
Doc with ID '43' has been processed before.
Doc with ID '44' has been processed before.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 1664 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 2776 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 1749 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 2033 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 1704 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 1958 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 2672 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 2242 total tokens.
DEBUG: Getting embedding from OpenAI
DEBUG: Got embedding. Usage: 1714 total tokens.
DEBUG: Getting 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


def kmeans_cluster_embeddings(df: pd.DataFrame, column_name: str, n_clusters: int = 108) -> pd.DataFrame:
    """
    Applies KMeans clustering to a column of document embeddings in a DataFrame.

    Parameters:
    - df: pandas DataFrame containing the embeddings.
    - column_name: str, name of the column that contains embeddings (as list/array).
    - n_clusters: int, number of clusters to form (default is 108).

    Returns:
    - df: original DataFrame with an added column 'cluster' containing cluster labels.
    """
    # Extract embeddings and convert to NumPy array
    embeddings = np.vstack(df[column_name].values)

    # Run KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)

    # Add the labels back to the DataFrame
    df["cluster"] = cluster_labels

    return df